### 1. Import Libraries

In [67]:
# Data manipulation
import pandas as pd
import numpy as np

# Date handling
from datetime import datetime

# ML preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Display all columns in notebook
pd.set_option('display.max_columns', None)

### 2. Load datasets

In [68]:
# Load data from 'data' folder
users = pd.read_csv('data/user_data.csv')
subscriptions = pd.read_csv('data/subscriptions.csv')
plans = pd.read_csv('data/subscription_plans.csv')
logs = pd.read_csv('data/subscription_logs.csv')
billing = pd.read_csv('data/billing_information.csv')

# Quick preview
print("Users Dataset:")
display(users.head())

print("Subscriptions Dataset:")
display(subscriptions.head())

print("Plans Dataset:")
display(plans.head())

print("Subscription Logs:")
display(logs.head())

print("Billing Info:")
display(billing.head())


Users Dataset:


,User Id,Name,Phone,Email,Status
0,1,User1,1234567801,user1@example.com,active
1,2,User2,1234567802,user2@example.com,inactive
2,3,User3,1234567803,user3@example.com,inactive
3,4,User4,1234567804,user4@example.com,active
4,5,User5,1234567805,user5@example.com,inactive


Subscriptions Dataset:


,Subscription Id,subscription_type,Product Id,User Id,Status,Start Date,Last Billed Date,Last Renewed Date,Terminated Date,Grace Time
0,1,monthly,46,61,PAUSED,2024-04-20,2024-01-20,2024-09-06,NaN,5
1,2,monthly,35,37,active,2024-08-24,2024-03-19,2024-10-29,NaN,5
2,3,monthly,96,24,active,2024-01-26,2023-12-10,2024-09-04,NaN,5
3,4,monthly,79,69,active,2024-07-31,2024-11-21,2024-07-30,NaN,5
4,5,monthly,52,3,active,2024-07-23,2024-04-05,2024-10-08,NaN,5


Plans Dataset:


,Product Id,Name,Price,Auto Renewal Allowed,Status
0,1,Plan1,57.65,No,Active
1,2,Plan2,15.30,Yes,Active
2,3,Plan3,73.86,Yes,Active
3,4,Plan4,27.82,Yes,Active
4,5,Plan5,42.58,Yes,Active


Subscription Logs:


,Subscription id,current status,next status,action,action date
0,30,initialized,paused,renew,2024-05-17
1,51,active,paused,billing_success,2024-10-28
2,15,active,paused,renew,2024-09-18
3,69,active,paused,renew_failed,2024-08-09
4,99,initialized,active,renew_failed,2024-07-23


Billing Info:


,billing_id,subscription_id,amount,billing_date,payment_status,Unnamed: 5,Unnamed: 6
0,1,54,308.56,2024-01-04,paid,NaN,NaN
1,2,52,200.11,2024-05-26,paid,NaN,NaN
2,3,6,407.49,2024-06-24,paid,NaN,NaN
3,4,100,289.68,2024-04-09,pending,NaN,NaN
4,5,16,114.07,2024-11-22,pending,NaN,NaN


### 3. Data Overview

In [69]:
# Check for missing values
print("Missing values per dataset:")
print("Users:", users.isnull().sum())
print("Subscriptions:", subscriptions.isnull().sum())
print("Plans:", plans.isnull().sum())
print("Logs:", logs.isnull().sum())
print("Billing:", billing.isnull().sum())

# Data types
print("\nData types:")
print(users.dtypes)
print(subscriptions.dtypes)

Missing values per dataset:
Users: User Id    0
Name       0
Phone      0
Email      0
Status     0
dtype: int64
Subscriptions: Subscription Id        0
subscription_type      0
Product Id             0
User Id                0
Status                 0
Start Date             0
Last Billed Date       0
Last Renewed Date      0
Terminated Date      100
Grace Time             0
dtype: int64
Plans: Product Id              0
Name                    0
Price                   0
Auto Renewal Allowed    0
Status                  0
dtype: int64
Logs: Subscription id    0
current status     0
next status        0
action             0
action date        0
dtype: int64
Billing: billing_id           0
subscription_id      0
amount               0
billing_date         0
payment_status       0
Unnamed: 5         100
Unnamed: 6          99
dtype: int64

Data types:
User Id     int64
Name       object
Phone       int64
Email      object
Status     object
dtype: object
Subscription Id        int64
subscr

### 4. Data Cleaning
Steps:

Fill missing values if needed.

Convert date columns to datetime.

Remove duplicates if any.

In [70]:
# Convert date columns to datetime
date_cols_sub = ['Start Date', 'Last Billed Date', 'Last Renewed Date', 'Terminated Date']
for col in date_cols_sub:
    if col in subscriptions.columns:
        subscriptions[col] = pd.to_datetime(subscriptions[col], errors='coerce')

if 'action_date' in logs.columns:
    logs['action_date'] = pd.to_datetime(logs['action_date'], errors='coerce')
if 'billing_date' in billing.columns:
    billing['billing_date'] = pd.to_datetime(billing['billing_date'], errors='coerce')

# Fill missing numeric values with 0 (example: Grace Time)
if 'Grace Time' in subscriptions.columns:
    subscriptions['Grace Time'] = subscriptions['Grace Time'].fillna(0)

# Remove duplicates if any
users.drop_duplicates(inplace=True)
subscriptions.drop_duplicates(inplace=True)
plans.drop_duplicates(inplace=True)
logs.drop_duplicates(inplace=True)
billing.drop_duplicates(inplace=True)

print("Data cleaning completed.")

Data cleaning completed.


### 5. Feature Engineering
We’ll generate some features that will be useful for recommendations and churn prediction:

Subscription Duration

Days Since Last Renewal

Payment Success Rate

Active Subscription Count per User

In [71]:
# Subscription duration (days)
subscriptions['subscription_duration'] = (subscriptions['Last Renewed Date'] - subscriptions['Start Date']).dt.days

# Days since last renewal
subscriptions['days_since_last_renewal'] = (pd.to_datetime('2025-09-13') - subscriptions['Last Renewed Date']).dt.days

# Payment success rate per user
payment_summary = billing.groupby('subscription_id')['payment_status'].value_counts(normalize=True).unstack(fill_value=0)
payment_summary['success_rate'] = payment_summary.get('paid', 0)
subscriptions = subscriptions.merge(payment_summary['success_rate'], left_on='Subscription Id', right_index=True, how='left')

# Active subscription count per user
active_count = subscriptions[subscriptions['Status'] == 'active'].groupby('User Id').size().reset_index(name='active_subscription_count')
users = users.merge(active_count, left_on='User Id', right_on='User Id', how='left')
users['active_subscription_count'] = users['active_subscription_count'].fillna(0)

display(subscriptions.head())
display(users.head())


,Subscription Id,subscription_type,Product Id,User Id,Status,Start Date,Last Billed Date,Last Renewed Date,Terminated Date,Grace Time,subscription_duration,days_since_last_renewal,success_rate
0,1,monthly,46,61,PAUSED,2024-04-20,2024-01-20,2024-09-06,NaT,5,139,372,0.0
1,2,monthly,35,37,active,2024-08-24,2024-03-19,2024-10-29,NaT,5,66,319,NaN
2,3,monthly,96,24,active,2024-01-26,2023-12-10,2024-09-04,NaT,5,222,374,0.0
3,4,monthly,79,69,active,2024-07-31,2024-11-21,2024-07-30,NaT,5,-1,410,0.0
4,5,monthly,52,3,active,2024-07-23,2024-04-05,2024-10-08,NaT,5,77,340,1.0


,User Id,Name,Phone,Email,Status,active_subscription_count
0,1,User1,1234567801,user1@example.com,active,0.0
1,2,User2,1234567802,user2@example.com,inactive,1.0
2,3,User3,1234567803,user3@example.com,inactive,2.0
3,4,User4,1234567804,user4@example.com,active,0.0
4,5,User5,1234567805,user5@example.com,inactive,0.0


### 6. Data Transformation
Encode categorical variables.

Scale numerical variables if needed for ML models.

In [72]:
# ai_notebook.ipynb - Cell 6

# Label encode categorical features
label_cols = ['Status', 'subscription_type']
le = LabelEncoder()
for col in label_cols:
    subscriptions[col] = le.fit_transform(subscriptions[col])

# Scale numerical features
num_cols = ['subscription_duration', 'days_since_last_renewal', 'success_rate', 'Grace Time']
scaler = StandardScaler()
subscriptions[num_cols] = scaler.fit_transform(subscriptions[num_cols])

display(subscriptions.head())

,Subscription Id,subscription_type,Product Id,User Id,Status,Start Date,Last Billed Date,Last Renewed Date,Terminated Date,Grace Time,subscription_duration,days_since_last_renewal,success_rate
0,1,0,46,61,0,2024-04-20,2024-01-20,2024-09-06,NaT,0.0,0.786428,-0.780252,-0.748867
1,2,0,35,37,1,2024-08-24,2024-03-19,2024-10-29,NaT,0.0,0.264858,-1.285424,NaN
2,3,0,96,24,1,2024-01-26,2023-12-10,2024-09-04,NaT,0.0,1.379446,-0.761189,-0.748867
3,4,0,79,69,1,2024-07-31,2024-11-21,2024-07-30,NaT,0.0,-0.213844,-0.418053,-0.748867
4,5,0,52,3,1,2024-07-23,2024-04-05,2024-10-08,NaT,0.0,0.343451,-1.085261,1.818676


### 7. Save Cleaned Data

In [81]:
# Save cleaned datasets for ML model training
subscriptions.to_csv('data/subscriptions_cleaned.csv', index=False)
users.to_csv('data/users_cleaned.csv', index=False)

print("Cleaned datasets saved for next milestones.")

Cleaned datasets saved for next milestones.


## Phase 2: Model Development and Evaluation

#### 1. Churn Prediction

Step1: Prepare the Data

In [73]:
# Define target variable: churn
# Assume Status 0 = inactive/paused, 1 = active
subscriptions['churn'] = subscriptions['Status'].apply(lambda x: 1 if x==0 else 0)  

# Features for model
features = ['subscription_duration', 'days_since_last_renewal', 'success_rate', 'Grace Time', 'subscription_type']
X = subscriptions[features]
y = subscriptions['churn']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Step2: Train a Random Forest Classifier

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Initialize model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluation
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Churn Prediction Metrics:")
print(f"Accuracy: {acc:.2f}, Precision: {prec:.2f}, Recall: {rec:.2f}, F1-Score: {f1:.2f}")
print("Confusion Matrix:")
print(cm)

Churn Prediction Metrics:
Accuracy: 0.45, Precision: 0.62, Recall: 0.38, F1-Score: 0.48
Confusion Matrix:
[[4 3]
 [8 5]]


#### 2. Plan Recommendation System
We’ll use content-based filtering for simplicity since datasets are small:

Step 1: Prepare User-Plan Matrix

In [75]:
# Merge subscriptions with plans
sub_plan = subscriptions.merge(plans, left_on='Product Id', right_on='Product Id', how='left')

# Content features: Price, Auto Renewal, etc.
sub_plan['auto_renewal'] = sub_plan['Auto Renewal Allowed'].apply(lambda x: 1 if x=='Yes' else 0)
sub_plan_features = sub_plan[['User Id', 'Product Id', 'Price', 'auto_renewal']]

# Pivot table: users vs products
user_product_matrix = sub_plan_features.pivot_table(index='User Id', columns='Product Id', values='Price', fill_value=0)
display(user_product_matrix.head())

Product Id,1,2,4,6,8,10,11,13,14,17,20,21,22,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,44,45,46,49,50,51,52,53,57,58,59,62,63,64,65,69,70,71,73,74,75,76,77,78,79,81,82,88,90,91,92,93,94,96,98,99
User Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.87,0.0,76.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,43.4,0.00,0.0,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,56.62,0.0,0.0,0.0
7,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.65,0.0,0.0,0.00,0.0,0.0,0.0
8,0.0,0.0,0.0,95.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0


Step 2: Compute Recommendations (Cosine Similarity)

In [84]:
def recommend_plans(user_id, top_n=3):
    if user_id not in user_similarity_df.index:
        # Fallback: recommend most popular plans overall
        popular_plans = sub_plan_features['Product Id'].value_counts().head(top_n).index.tolist()
        return popular_plans
    
    # Find most similar users
    sim_users = user_similarity_df[user_id].sort_values(ascending=False)[1:4].index
    
    # Collect plans subscribed by similar users
    recommended = sub_plan_features[sub_plan_features['User Id'].isin(sim_users)]['Product Id'].value_counts()
    
    # Remove plans already subscribed by the current user
    user_plans = sub_plan_features[sub_plan_features['User Id']==user_id]['Product Id']
    recommended = recommended[~recommended.index.isin(user_plans)]
    
    # If nothing left, fallback to popular plans
    if recommended.empty:
        return sub_plan_features['Product Id'].value_counts().head(top_n).index.tolist()
    
    return recommended.head(top_n).index.tolist()

# Example usage
print("Recommendations for User 1:", recommend_plans(1))
print("Recommendations for User 24:", recommend_plans(24))

Recommendations for User 1: [79, 70, 63]
Recommendations for User 24: [94, 93, 70]


## Save Models and Matrices

In [86]:
# Save churn model
import joblib
joblib.dump(rf_model, 'models/churn_model.pkl')

# Save user-product matrix for recommendations
joblib.dump(user_product_matrix, 'models/user_product_matrix.pkl')

['models/user_product_matrix.pkl']